<a href="https://colab.research.google.com/github/SaraWestWA/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module1-rnn-and-lstm/SW_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [33]:
import requests
import pandas as pd

from __future__ import print_function

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, SimpleRNN, LSTM
from tensorflow.keras.datasets import imdb

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [72]:
import re
url = r"https://raw.githubusercontent.com/SaraWestWA/DS-Unit-4-Sprint-3-Deep-Learning/main/module1-rnn-and-lstm/Shakespeare_Sonnets.txt"
!wget $url
file_obj = open("Shakespeare_Sonnets.txt", "r")
text1 = file_obj.read()
re.findall("\n\s*([a-zA-Z].*)", text1)

--2021-02-11 04:34:05--  https://raw.githubusercontent.com/SaraWestWA/DS-Unit-4-Sprint-3-Deep-Learning/main/module1-rnn-and-lstm/Shakespeare_Sonnets.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101880 (99K) [text/plain]
Saving to: ‘Shakespeare_Sonnets.txt.2’

Shakespeare_Sonnets 100%[===================>]  99.49K  --.-KB/s    in 0.008s  

2021-02-11 04:34:05 (12.3 MB/s) - ‘Shakespeare_Sonnets.txt.2’ saved [101880/101880]



['From fairest creatures we desire increase,',
 'That thereby beauty’s rose might never die,',
 'But as the riper should by time decease,',
 'His tender heir might bear his memory:',
 'But thou contracted to thine own bright eyes,',
 'Feed’st thy light’s flame with self-substantial fuel,',
 'Making a famine where abundance lies,',
 'Thy self thy foe, to thy sweet self too cruel:',
 'Thou that art now the world’s fresh ornament,',
 'And only herald to the gaudy spring,',
 'Within thine own bud buriest thy content,',
 'And, tender churl, mak’st waste in niggarding:',
 'Pity the world, or else this glutton be,',
 'To eat the world’s due, by the grave and thee.',
 'When forty winters shall besiege thy brow,',
 'And dig deep trenches in thy beauty’s field,',
 'Thy youth’s proud livery so gazed on now,',
 'Will be a tattered weed of small worth held:',
 'Then being asked, where all thy beauty lies,',
 'Where all the treasure of thy lusty days;',
 'To say, within thine own deep sunken eyes,',

In [97]:
text = text1

In [98]:
# clean data
text = text.lower()
text = text.replace('\n', '')
clean_text = ''.join(i for i in text if not i.isdigit())
clean_text = re.sub(r"[^A-Za-z .!?,]+", ' ', clean_text)
clean_text  = ' '.join(x for x in clean_text.split())

In [99]:
# Encode data as characters
chars=list(set(clean_text))
len(chars)

31

In [100]:
# create lookup tables
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

In [101]:
# create sequenced data
maxlen = 40
step = 5 # slide text window over 5 characters

encoded = [char_int[c] for c in clean_text]
sequences = [] # stores the 40 length elements
next_char = [] # one next character for each sequence

for i in range(0, len(encoded) - maxlen, step):
  sequences.append(encoded[i : i + maxlen])
  next_char.append(encoded[i + maxlen])

print ('Number of Sequences: ', len(sequences))

Number of Sequences:  18358


In [102]:
# look at first sequence
clean_text[0:40]

'the sonnets from fairest creatures we de'

In [103]:
# create x & y
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
    y[i, next_char[i]] = 1
    
print(x.shape, y.shape)

(18358, 40, 31) (18358, 31)


In [104]:
x[0]

array([[False, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False]])

In [105]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False])

In [112]:
# make a single LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

#compile
model.compile(loss='categorical_crossentropy',
              optimizer='adam')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 64)                24576     
_________________________________________________________________
dense_4 (Dense)              (None, 31)                2015      
Total params: 26,591
Trainable params: 26,591
Non-trainable params: 0
_________________________________________________________________


In [128]:
# create a function to sample an index to use to begin our generated text
def sample(preds):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / 1
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [132]:
# function to print the text generated in each epoch
def print_gen_text(epoch, _):
  print()
  print('----- Generating text after Epoch: %d' % epoch)
  
  start_index = random.randint(0, len(clean_text) - maxlen - 1)
  
  generated = ''
  sentence = clean_text[start_index: start_index + maxlen]
  generated += sentence
  
  print('----- Generating with seed: "' + sentence + '"')
  sys.stdout.write(generated)
    
  for i in range(400):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_int[char]] = 1
          
      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds)
      next_char = int_char[next_index]
      
      sentence = sentence[1:] + next_char
      
      sys.stdout.write(next_char)
      sys.stdout.flush()
  print()

print_callback = LambdaCallback(on_epoch_end=print_gen_text)


In [133]:
# Fit model

model.fit (x, y,
           batch_size=32,
           epochs=8,
           callbacks=[print_callback])

Epoch 1/8
574/574 [==============================] - 10s 17ms/step - loss: 1.8681

----- Generating text after Epoch: 0
----- Generating with seed: "h gems with april s first born flowers a"
h gems with april s first born flowers ade hare sell thy gratm, tf riwh not ones indtteran treeers mist swaip ssearts st an inel she arm dily unirith!fron i hils,an the  ackn sherr.eddy bariod dath to tet my seaf m ye. lima nesess a toute what whis reakty i wat your s deright leosirn oo si rurst sasth not umbenene thye swure mighing in hing eats bet hre, that worecs the vift swell degiind ared i eris as hice love leest une, on tenane di
Epoch 2/8
574/574 [==============================] - 10s 17ms/step - loss: 1.8507

----- Generating text after Epoch: 1
----- Generating with seed: "hat s new to speak, what now to register"
hat s new to speak, what now to register wornte,an yok. or lodatakl shou, sut eonty, mud to conded swear bewprenust,shame, and heald noves sonend,bivest to thy form bet,and sash

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN